In [16]:
#import libraries
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow

In [10]:
train_data_path = '/content/drive/My Drive/Face_mask_detection/dataset/train'
validation_data_path = '/content/drive/My Drive/Face_mask_detection/dataset/valid'

In [11]:
training_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')
 

training_data = training_datagen.flow_from_directory(train_data_path, # this is the target directory
                                      target_size=(200, 200), # all images will be resized to 150x150
                                      batch_size=128,
                                      class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

Found 3649 images belonging to 2 classes.


In [12]:
training_data.class_indices

{'with_mask': 0, 'without_mask': 1}

In [13]:
valid_datagen = ImageDataGenerator(rescale=1./255)
 
# this is a similar generator, for validation data
valid_data = valid_datagen.flow_from_directory(validation_data_path,
                                  target_size=(200,200),
                                  batch_size=128,
                                  class_mode='binary')
 

 
# save best model using vall accuracy
model_path = '/content/drive/My Drive/Face_mask_detection/Models/Face_mask_scratch.h5'
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

Found 806 images belonging to 2 classes.


In [14]:
#Building cnn model
cnn_model = keras.models.Sequential([
                                    keras.layers.Conv2D(filters=32, kernel_size=5, input_shape=[200, 200, 3]),
                                    keras.layers.MaxPooling2D(pool_size=(4,4)),
                                    keras.layers.Conv2D(filters=64, kernel_size=4),
                                    keras.layers.MaxPooling2D(pool_size=(3,3)),
                                    keras.layers.Conv2D(filters=128, kernel_size=3),
                                    keras.layers.MaxPooling2D(pool_size=(2,2)),                                    
                                    keras.layers.Conv2D(filters=256, kernel_size=2),
                                    keras.layers.MaxPooling2D(pool_size=(2,2)),
 
                                    keras.layers.Dropout(0.5),                                                                        
                                    keras.layers.Flatten(), # neural network beulding
                                    keras.layers.Dense(units=128, activation='relu'), # input layers
                                    keras.layers.Dropout(0.1),                                    
                                    keras.layers.Dense(units=256, activation='relu'),                                    
                                    keras.layers.Dropout(0.25),                                    
                                    keras.layers.Dense(units=2, activation='softmax') # output layer
])

In [15]:
# compile cnn model
cnn_model.compile(optimizer = Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
# train cnn model
history = cnn_model.fit(training_data, 
                          epochs=50, 
                          verbose=1, 
                          validation_data= valid_data,
                          callbacks=callbacks_list) # time start 14.25

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch 1/50
29/29 [==============================] - ETA: 0s - loss: 0.4875 - accuracy: 0.7616 
Epoch 00001: val_accuracy improved from -inf to 0.92556, saving model to /content/drive/My Drive/Face_mask_detection/Models/Face_mask_scratch.h5
29/29 [==============================] - 2040s 70s/step - loss: 0.4875 - accuracy: 0.7616 - val_loss: 0.2137 - val_accuracy: 0.9256
Epoch 2/50
29/29 [==============================] - ETA: 0s - loss: 0.3293 - accuracy: 0.8791
Epoch 00002: val_accuracy improved from 0.92556 to 0.93672, saving model to /content/drive/My Drive/Face_mask_detection/Models/Face_mask_scratch.h5
29/29 [==============================] - 47s 2s/step - loss: 0.3293 - accuracy: 0.8791 - val_loss: 0.1945 - val_accuracy: 0.9367
Epoch 3/50
29/29 [==============================] - ETA: 0s - loss: 0.3209 - accuracy: 0.8772
Epoch 00003: val_accuracy improved from 0.93672 to 0.94045, saving model to /content/drive/My Drive/Face_mask_detection/Models/Face_mask_scratch.h5
29/29 [========

In [17]:
cnn_model.save('/content/drive/My Drive/Face_mask_detection/Models/final_Face_mask_scratch.h5')